In [1]:
from benchmark import impl
from benchmark import spec
from decimal import Decimal
import pandas as pd

Order = spec.Order
tradeBySourceAmount = spec.tradeBySourceAmount
tradeByTargetAmount = spec.tradeByTargetAmount

Set up ETH order parameters with 5 liquidity and yint 10

In [2]:
order_params = {
    'liquidity':Decimal('5'),
    'highestRate':Decimal('0.0005'), # 2000
    'lowestRate':Decimal('0.0004'),  # 2500
    'marginalRate':Decimal('0.0004486068'),
    }

In [3]:
orders = []
for i in range(5):
    orders += [Order(order_params)]
orders[0].z

Decimal('9.9999995499958147428185172765306785359256132499398')

In [4]:
indexes = list(range(len(orders)))

In [5]:
total_liquidity = sum(o.y for o in orders)
total_liquidity

Decimal('25')

# Trade Functions
- Note they use pure math so will exceed the liquidity of an order

In [6]:
# trade 5000 USDC into the first order 
tradeBySourceAmount(5000, orders[0])

(5000, Decimal('2.1883258511228429259642260584200661234770595532491'))

In [7]:
# trade out exactly 2.188... ETH
tradeByTargetAmount(Decimal('2.1883258511228429259642260584200661234770595532491'), orders[0])

(Decimal('5000.0000000000000000000000000000000000000000000001'),
 Decimal('2.1883258511228429259642260584200661234770595532491'))

# Match by Target
1. hypothetical_output_amt for full inputAmount

In [8]:
# total liquidity is 25 ETH
inputAmount = 20

hypothetical_output_amts = {
            i: tradeByTargetAmount(x=inputAmount, order=orders[i])[0] for i in indexes
        }

hypothetical_output_amts

{0: Decimal('57371.228193369793945368184469586874650176609516568'),
 1: Decimal('57371.228193369793945368184469586874650176609516568'),
 2: Decimal('57371.228193369793945368184469586874650176609516568'),
 3: Decimal('57371.228193369793945368184469586874650176609516568'),
 4: Decimal('57371.228193369793945368184469586874650176609516568')}

In [9]:
ordered_amts = {
    j: hypothetical_output_amts[j]
    for j in sorted(
        indexes, key=lambda i: hypothetical_output_amts[i], reverse=True
    )
}
ordered_amts

{0: Decimal('57371.228193369793945368184469586874650176609516568'),
 1: Decimal('57371.228193369793945368184469586874650176609516568'),
 2: Decimal('57371.228193369793945368184469586874650176609516568'),
 3: Decimal('57371.228193369793945368184469586874650176609516568'),
 4: Decimal('57371.228193369793945368184469586874650176609516568')}

In [10]:
ordered_associated_liquidity = {i:orders[i].y for i in ordered_amts.keys()}
ordered_associated_liquidity

{0: Decimal('5'),
 1: Decimal('5'),
 2: Decimal('5'),
 3: Decimal('5'),
 4: Decimal('5')}

In [11]:
results = pd.DataFrame(
    [
        hypothetical_output_amts.keys(),
        hypothetical_output_amts.values(),
    ],
    index=[
        "indexes",
        "hypothetical_output_amts",
    ],
)
results = results.T.copy()
results.sort_values(
    by=["hypothetical_output_amts", 'indexes'], ascending=[False, True], inplace=True
)

results2 = pd.DataFrame(
    [
        ordered_associated_liquidity.keys(),
        ordered_associated_liquidity.values(),
    ],
    index=[
        "ordered_associated_liquidity_keys",
        "ordered_associated_liquidity",
    ],
)
results2 = results2.T.copy()

results = pd.merge(results, results2, how='left', left_on = 'indexes', right_on='ordered_associated_liquidity_keys')

results.fillna(0, inplace=True)
results.reset_index(inplace=True, drop=True)
results

,indexes,hypothetical_output_amts,ordered_associated_liquidity_keys,ordered_associated_liquidity
0,0,57371.2281933697939453681844695868746501766095...,0,5
1,1,57371.2281933697939453681844695868746501766095...,1,5
2,2,57371.2281933697939453681844695868746501766095...,2,5
3,3,57371.2281933697939453681844695868746501766095...,3,5
4,4,57371.2281933697939453681844695868746501766095...,4,5
